In [ ]:
import pandas as pd
import numpy as np
np.seterr(all='ignore')
import matplotlib.pyplot as plt
from hydroeval import evaluator, nse, kge, pbias
from tqdm import tqdm
import datetime as dt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_SS.csv")

colnm = ["time",'obs','model','water_year']
out_col = ['gage','model','nse','kge','kge_r','kge_a','kge_b','n_wateryear']
# df_list = ['vic','prms','nwm0','nwm1']
# df_list = [VIC,PRMS,NWM0,NWM1]

out_df = pd.DataFrame(columns = out_col)

# i = 1
for i in tqdm(range(0,len(pnwNP))):
# for i in tqdm(range(0,2)):

    try:
        ## Read in streamflow and simulation dataset
        df = pd.read_csv('../data/pnwNP_modeledData_NWM3/'+str(pnwNP['gage'][i])+".csv")
        df['time'] = pd.to_datetime(df['time'])
        df['water_year'] = df.time.dt.year.where(df.time.dt.month < 10, df.time.dt.year + 1)


        ## subset the dataset to only overlapping time periods
        VIC = df[(df['streamflow_NWIS'].notna())&(df['streamflow_VIC'].notna())][['time','streamflow_NWIS','streamflow_VIC','water_year']].reset_index(drop=True)
        VIC.columns = colnm
        VIC.name = "vic"

        PRMS = df[(df['streamflow_NWIS'].notna())&(df['streamflow_PRMS'].notna())][['time','streamflow_NWIS','streamflow_PRMS','water_year']].reset_index(drop=True)
        PRMS.columns = colnm
        PRMS.name = "prms"

        NWM0 = df[(df['streamflow_NWIS'].notna())&(df['streamflow_NWM2d0'].notna())][['time','streamflow_NWIS','streamflow_NWM2d0','water_year']].reset_index(drop=True)
        NWM0.columns = colnm
        NWM0.name = "nwm0"

        NWM1 = df[(df['streamflow_NWIS'].notna())&(df['streamflow_NWM2d1'].notna())][['time','streamflow_NWIS','streamflow_NWM2d1','water_year']].reset_index(drop=True)
        NWM1.columns = colnm
        NWM1.name = "nwm1"
        
        NWM3 = df[(df['streamflow_NWIS'].notna())&(df['streamflow_NWM3'].notna())][['time','streamflow_NWIS','streamflow_NWM3','water_year']].reset_index(drop=True)
        NWM3.columns = colnm
        NWM3.name = "nwm3"

        temp = NWM3

        df_list = [VIC,PRMS,NWM0,NWM1,NWM3]

        for j, df in enumerate(df_list):
            temp = pd.DataFrame(df_list[j])
            ## Calculate hydrovariables
            NSE = evaluator(nse,
                            simulations = temp['model'],
                            evaluation = temp['obs'], 
                            axis=1,
                            transform='inv',
                            epsilon=0.00001)

            KGE = evaluator(kge,
                            simulations = temp['model'],
                            evaluation = temp['obs'], 
                            axis=1,
                            transform='inv',
                            epsilon=0.00001)

            gage = pnwNP['gage'][i]


            # ## Return hydrovaribles and write to list
            out = pd.DataFrame(np.column_stack([int(gage),df_list[j].name,NSE,KGE,len(np.unique(temp["water_year"]))]))
            out.columns = out_col

            out_df = pd.concat([out_df,out])

    except:
        print("No data")
        pass
    

In [ ]:
dat = pd.read_csv("../data/pnwNP_StatsLocations.csv")
out_df["n_wateryear"] = out_df["n_wateryear"].astype(int)
tt = out_df
tt = tt.reset_index(drop=True)
tt['gage']=tt['gage'].astype(np.int64)
tt = tt.merge(dat, how='inner', on='gage')

In [ ]:
tt.to_csv("../data/pnwNP_modelStats_NewEps_NWM3.csv")